# Data Cleaning with gpt-3.5-turbo of openai
In this Notebook, we will make use of Chatgpt gpt-3.5-turbo api to apply data cleaning.
Each line of the dates.txt file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset, and compare performance results of this artifical intelligence tool.



In [14]:
#First, let's import some libraries, open our dataset and print some examples.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import regex as re
import random
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
list_=[]
list_=list(df.values)
len(list_)
df.tail(10)

490                                  Lab: B12 969 2007\n
491                                   )and 8mo in 2009\n
492    .Moved to USA in 1986. Suffered from malnutrit...
493                                              r1978\n
494    . Went to Emerson, in Newfane Alaska. Started ...
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
dtype: object

Now, let's import openai and define our api_key, this key can be found in your account settings in openai.com

In [3]:
import openai
openai.api_key  ='sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

*get_completion function uses gpt-3.5-turbo to get a response of the model sending only the prompt with the instructions

In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

Now, let's create a foor loop to iterate over the sentences in the dataset, each sentence in the list will be used as the 
text for gpt to extract the dates. 
The prompt is a simple paragraph of 4 lines, where we inidcate where the text is, what to do whith this text, and how do
we need the results.
For this example we will pass in batches of 50 sentences, and we will only use half of the dataset, so we will call gpt just
five times. 
The purpose of this calls are to extract the date of each sentence, we will end up with a appended list with all the dates
extracted.

In [5]:
delimiter_d=0
delimiter_up=0
dates=[]
start_time = time.time()
for i in range(5):
    delimiter_d=delimiter_up
    delimiter_up=50*(i+1)

    text=list_[delimiter_d:delimiter_up]
    prompt = f"""
        Extract the dates of each sentence in the text delimited with triple backticks,  \
        if the day or month is not found, complete the date with day 01 and month 01, there are 50 dates\
        in the text, give me the results in a list.
        ```{text}```
        """
    response = get_completion(prompt)
    print(response)
    dates.append(response)
end_time = time.time()
execution_time = end_time - start_time
#print(dates)
print(execution_time)


['03/25/93', '6/18/85', '7/8/71', '9/27/75', '2/6/96', '7/06/79', '5/18/78', '10/24/89', '3/7/86', '4/10/71', '5/11/85', '4/09/75', '8/01/98', '1/26/72', '5/24/1990', '1/25/2011', '4/12/82', '10/13/1976', '4/24/98', '5/21/77', '7/21/98', '10/21/... '9/22/89', '9/02/76', '9/12/71', '10/24/86', '03/31/1985', '7/20/72', '4/12/87', '06/20/91', '5/12/2012', '3/15/83', '2/14/73', '5/24/88', '7/27/1986', '1-14-81', '7-29-75', '6/24/87', '8/14/94', '4/13/2002', '8/16/82', '2/15/1998', '7/15/91', '06/12/94', '9/17/84', '2/28/75']
['11/22/75', '5/24/91', '6/13/92', '7/11/71', '12/26/86', '10/11/1987', '3/14/95', '12/01/73', '12/5/2010', '08/20/1982', '7/24/95', '8/06/83', '02/22/92', '6/28/87', '07/29/1994', '08/11/78', '10/29/91', '7/6/91', '1/21/87', '11/3/1985', '7/04/82... '09/19/81', '9/6/79', '12/5/87', '01/05/1999', '4/22/80', '10/04/98', '6/29/81', '8/04/78', '7/07/1974', '09/14/2000', '5/18/71', '8/09/1981', '6/05/93', '8/9/97', '12/8/82', '8/26/89', '10/13/95', '4/19/91', '04/08/2004',

As can be seen, the results show the extracted dates. However, something happened in batch 3, where the extracted results were not only the date, they include some text.
The execution time of this code was 136 seconds, it is a elevated time for the extracted task in comparison of using regex, 
however, we just write 4 lines of a prompt insted of spending a considerable amount of time planning, writting and debugging 
a code using lists, pandas and regex.

Now, we will iterate once again to extract the dates and change them to a specific format, this time we will use the 
list of the results of the previous cell.

In [23]:
start_time = time.time()
text=dates
prompt = f"""
        The text delimited with triple backticks contains different dates, change them to the same format delimited by <>\ 
        and put them in a list
        <month/day/year>
        ```{text}```
        """
response = get_completion(prompt)
print(response)
end_time = time.time()
execution_time = end_time - start_time

print(execution_time)

<03/25/93>, <6/18/85>, <7/8/71>, <9/27/75>, <2/6/96>, <7/06/79>, <5/18/78>, <10/24/89>, <3/7/86>, <4/10/71>, <5/11/85>, <4/09/75>, <8/01/98>, <1/26/72>, <5/24/1990>, <1/25/2011>, <4/12/82>, <10/13/1976>, <4/24/98>, <5/21/77>, <7/21/98>, <10/21/... <9/22/89>, <9/02/76>, <9/12/71>, <10/24/86>, <03/31/1985>, <7/20/72>, <4/12/87>, <06/20/91>, <5/12/2012>, <3/15/83>, <2/14/73>, <5/24/88>, <7/27/1986>, <1-14-81>, <7-29-75>, <6/24/87>, <8/14/94>, <4/13/2002>, <8/16/82>, <2/15/1998>, <7/15/91>, <06/12/94>, <9/17/84>, <2/28/75>
<11/22/75>, <5/24/91>, <6/13/92>, <7/11/71>, <12/26/86>, <10/11/1987>, <3/14/95>, <12/01/73>, <12/5/2010>, <08/20/1982>, <7/24/95>, <8/06/83>, <02/22/92>, <6/28/87>, <07/29/1994>, <08/11/78>, <10/29/91>, <7/6/91>, <1/21/87>, <11/3/1985>, <7/04/82>, <... <09/19/81>, <9/6/79>, <12/5/87>, <01/05/1999>, <4/22/80>, <10/04/98>, <6/29/81>, <8/04/78>, <7/07/1974>, <09/14/2000>, <5/18/71>, <8/09/1981>, <6/05/93>, <8/9/97>, <12/8/82>, <8/26/89>, <10/13/95>, <4/19/91>, <04/08/2004>

In [22]:
dates_extracted=[]
date_pattern =r'<(.*?)>'
dates_extracted= re.findall(date_pattern,response)
random_indexes = [random.randint(0, 250 - 1) for _ in range(10)]
for i in random_indexes:
    print(dates_extracted[i])

July 25, 1998
12 March 2004
21 Oct 1977
12/04/87
18 August 1995
8/14/94
4/12/82
4/13/2002
29 Jan 1994
18 August 1975
